# Data Imputation

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, ParameterGrid, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

From the Data_Explotation notebook, there were four predictors with missing data:
- `job`
- `education`
- `device`
- `outcome_old`

# Import Data

In [ ]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_test = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

## Dummify `marital` and `outcome_old`

In [ ]:
campaign_ad = pd.get_dummies(campaign_ad, columns=['marital', 'outcome_old'])

## Train-Valid-Test Split

First, we split training and test, keeping 70% of our Training within `X_train` and 30% within `X_test`. From the 30% Test split, we split in half, so that `X_test` had 15% and `X_valid` had the same percentage.

In [ ]:
X = campaign_ad.drop(columns = 'subscription')
y = campaign_ad['subscription']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 12) # Test split
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 46) # Validation split

These splits can be seen below.

In [ ]:
print('Size of training set:', X_train.shape[0])
print('Size of validation set:', X_valid.shape[0])
print('Size of test set:', X_test.shape[0])

size of training set: 6266
size of validation set: 1343
size of test set: 1343


#  Device Imputation
## Train `device` 

We began with computing the missing values of `device` because it had the least amount of categories.

In [ ]:
# This is the data set that we need to predict and fuse with actual Training data for future imputation of `education`` and `job`
X_tr_dev = X_train.drop(columns=['job', 'education'])
X_tr_imp_dev = X_tr_dev[X_tr_dev['device'] == 'na'].drop(columns = 'device') 
X_tr_imp_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,
4359,44,17,6,24.05,1,0,-1,0,0,0,0.076980,0,1,0,0,1,0,0
5378,31,26,5,8.60,5,0,-1,0,0,1,0.068700,1,0,0,0,1,0,0
3095,49,5,6,0.45,3,0,-1,0,0,0,0.088029,0,1,0,0,1,0,0
8737,36,16,6,33.80,11,0,-1,0,0,1,0.075291,0,1,0,0,1,0,0
7745,32,12,5,13.95,3,0,-1,0,0,1,0.085014,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510,32,6,6,29.80,5,0,-1,0,0,1,0.061255,0,1,0,0,1,0,0
4515,36,11,6,5.35,2,0,-1,0,0,0,0.095655,0,1,0,0,1,0,0
4030,29,6,5,12.90,2,0,-1,0,0,1,0.073194,0,1,0,0,1,0,0


In [ ]:
X_tr_dev = X_tr_dev[X_tr_dev['device'] != 'na'] # The data set that does not have NAs
y_tr_dev = X_tr_dev[['device']] 
X_tr_dev = X_tr_dev.drop(columns = 'device')
X_tr_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,
4949,59,6,8,3.90,2,0,-1,0,0,0,0.073675,0,1,0,0,1,0,0
4298,50,5,8,27.15,1,0,-1,0,0,0,0.093803,0,1,0,0,1,0,0
6904,55,21,11,2.25,5,0,-1,1,0,1,0.085450,0,1,0,0,1,0,0
4642,44,19,11,17.65,2,0,-1,0,0,0,0.086812,0,1,0,0,1,0,0
2403,53,19,11,27.55,1,0,-1,0,0,0,0.078133,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,27,4,2,53.40,1,0,-1,0,0,1,0.081256,0,0,1,0,1,0,0
278,42,12,5,0.70,1,0,-1,0,0,0,0.072803,0,1,0,0,1,0,0
3714,39,13,5,98.25,5,0,-1,1,0,1,0.072803,0,0,1,0,1,0,0


In [ ]:
# Transform categorical features into ordinal integers
enc = OrdinalEncoder(categories = [['smartphone', 'desktop']]) # smartphone = 0, desktop = 1
y_tr_dev = enc.fit_transform(y_tr_dev)
y_tr_dev

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [ ]:
# Changing shape of y_tr_dev so RandomForestClassifier properly works
print(y_tr_dev.shape)
y_tr_dev.shape = (y_tr_dev.shape[0],)
print(y_tr_dev.shape)

(4830, 1)
(4830,)


We utilize the `RandomForstClassifier` to input missing data for `device`.

In [ ]:
rfc_dev = RandomForestClassifier(n_estimators = 100, random_state = 59, n_jobs = -2)

In [ ]:
rfc_dev.fit(X_tr_dev, y_tr_dev) # Fit on training, device

RandomForestClassifier(n_jobs=-2, random_state=59)

In [ ]:
y_tr_dev_pred = rfc_dev.predict(X_tr_dev) # Predict on training, device
accuracy_score(y_true = y_tr_dev, y_pred = y_tr_dev_pred) # Compare accuracy from training prediction to true values
#np.array(y_tr_dev_pred.shape)

1.0

## Validation `device`

Now, we apply the same logic for `device`'s missing data within Validation.

In [ ]:
# Drop the other two missing values (`job` and `education`) to fill in `device` and ordering it accordingly
X_v_dev = X_valid.drop(columns = ['job', 'education'])
X_v_imp_dev = X_v_dev[X_v_dev['device'] == 'na'].drop(columns = 'device')
X_v_dev = X_v_dev[X_v_dev['device'] != 'na']
y_v_dev = X_v_dev[['device']]
X_v_dev = X_v_dev.drop(columns='device')
enc = OrdinalEncoder(categories = [['smartphone', 'desktop']])
y_v_dev = enc.fit_transform(y_v_dev)

In [ ]:
y_v_dev_pred = rfc_dev.predict(X_v_dev)
accuracy_score(y_pred = y_v_dev_pred, y_true = y_v_dev) # Compare the predicted versus true values to recieve the accuracy score

0.913926499032882

We have obtained a 91.4% accuracy on the missing data for Validation.

## Test `device`

We repreat this process once more for the test missing values in `device`.

In [ ]:
# Drop the other two missing values (`job` and `education`) to fill in `device` and ordering it accordingly
X_te_dev = X_test.drop(columns=['job', 'education'])
X_te_imp_dev = X_te_dev[X_te_dev['device'] == 'na'].drop(columns = 'device')
X_te_dev = X_te_dev[X_te_dev['device'] != 'na']
y_te_dev = X_te_dev[['device']]
X_te_dev = X_te_dev.drop(columns = 'device')
enc = OrdinalEncoder(categories = [['smartphone', 'desktop']])
y_te_dev = enc.fit_transform(y_te_dev)

In [ ]:
y_te_dev_pred = rfc_dev.predict(X_te_dev)
accuracy_score(y_pred = y_te_dev_pred, y_true = y_te_dev) # Compare the predicted versus true values to recieve the accuracy score

0.918111753371869

The accruacy slightly increases from the Validation score. Therefore, we will continue to use the `RandomForestClassifier()` to imput all the missing `device` data to input into our new Training set that will be composed of no missing values.

## Insert `device` Imputations in the Sets

In [ ]:
y_tr_imp_dev = rfc_dev.predict(X_tr_imp_dev)
y_v_imp_dev = rfc_dev.predict(X_v_imp_dev)
y_te_imp_dev = rfc_dev.predict(X_te_imp_dev)

In [ ]:
X_train.loc[X_train['device'] == 'smartphone', 'device'] = 0 # smartphone is considered 0 for train
X_train.loc[X_train['device'] == 'desktop', 'device'] = 1 # desktop is considered 1 for train
X_train.loc[X_tr_imp_dev.index, 'device'] = y_tr_imp_dev

In [ ]:
X_valid.loc[X_valid['device'] == 'smartphone', 'device'] = 0 # smartphone is considered 0 for valid
X_valid.loc[X_valid['device'] == 'desktop', 'device'] = 1 # desktop is considered 1 for valid
X_valid.loc[X_v_imp_dev.index, 'device'] = y_v_imp_dev

In [ ]:
X_test.loc[X_test['device'] == 'smartphone', 'device'] = 0 # smartphone is considered 0 for test
X_test.loc[X_test['device'] == 'desktop', 'device'] = 1 # desktop is considered 1 for test
X_test.loc[X_te_imp_dev.index, 'device'] = y_te_imp_dev

# Impute Most Common Value for `job` and `education`

In [ ]:
# Replace missing values with the mode of each of those variables
imputer = SimpleImputer(missing_values = "na", strategy = 'most_frequent')
X_train[['job', 'education']] = imputer.fit_transform(X_train[['job', 'education']])
X_valid[['job', 'education']] = imputer.fit_transform(X_valid[['job', 'education']])
X_test[['job', 'education']]  = imputer.fit_transform(X_test[['job', 'education']])

In [ ]:
# Create dummy variables for job, this will include 11 dummies
X_train = pd.get_dummies(X_train, columns = ['job'])
X_valid = pd.get_dummies(X_valid, columns = ['job'])
X_test  = pd.get_dummies(X_test, columns = ['job'])

In [ ]:
# Create ordinal values for education since they can be assumed to be ordered
enc = OrdinalEncoder(categories = [['high_school', 'university', 'grad_school']])
X_train['education'] = enc.fit_transform(X_train[['education']])
X_valid['education'] = enc.fit_transform(X_valid[['education']])
X_test['education']  = enc.fit_transform(X_test[['education']])

In [ ]:
# Review that all categories have been appropriately dummified 
X_train.columns

Index(['age', 'education', 'device', 'day', 'month', 'time_spent',
       'banner_views', 'banner_views_old', 'days_elapsed_old', 'X1', 'X2',
       'X3', 'X4', 'marital_divorced', 'marital_married', 'marital_single',
       'outcome_old_failure', 'outcome_old_na', 'outcome_old_other',
       'outcome_old_success', 'job_entrepreneur', 'job_freelance',
       'job_housekeeper', 'job_industrial_worker', 'job_manager',
       'job_retired', 'job_salesman', 'job_student', 'job_teacher',
       'job_technology', 'job_unemployed'],
      dtype='object')

In [ ]:
# Export the six splits into CSV for clean enviroment
X_train.to_csv('0_X_train.csv')
y_train.to_csv('0_y_train.csv')
X_valid.to_csv('1_X_valid.csv')
y_valid.to_csv('1_y_valid.csv')
X_test.to_csv('2_X_test.csv')
y_test.to_csv('2_y_test.csv')

# Prepping REAL Test Set

The code below follows the same procedure as throughout the Training, Validation, and Test sets, but now with the provided Test set for Kaggle submissions.

In [ ]:
# Dummify `marital` and `outcome_old`
campaign_test = pd.get_dummies(campaign_test, columns = ['marital', 'outcome_old'])

In [ ]:
# Drop the other two missing values (`job` and `education`) to fill in `device` and ordering it accordingly
X_campaign_test = campaign_test.drop(columns = ['job', 'education'])
X_imp_campaign_test = X_campaign_test[X_campaign_test['device'] == 'na'].drop(columns = 'device')
X_campaign_test = X_campaign_test[X_campaign_test['device'] != 'na']
y_campaign_test = X_campaign_test[['device']]
X_campaign_test = X_campaign_test.drop(columns = 'device')
enc = OrdinalEncoder(categories = [['smartphone', 'desktop']])
y_campaign_test = enc.fit_transform(y_campaign_test)

For `outcome_old`, we determined that `na` was meaningful and therefore, kept those values as is.

In [ ]:
y_campaign_test_pred = rfc_dev.predict(X_campaign_test)
accuracy_score(y_pred = y_campaign_test_pred, y_true = y_campaign_test) # Compare the predicted versus true values to recieve the accuracy score

0.9199591280653951

In [ ]:
y_imp_campaign_test = rfc_dev.predict(X_imp_campaign_test) # Predict on the test set to produce the response

In [ ]:
campaign_test.loc[campaign_test['device'] == 'smartphone', 'device'] = 0 # smartphone is considered 0 for real test
campaign_test.loc[campaign_test['device'] == 'desktop', 'device'] = 1 # desktop is considered 1 for real test
campaign_test.loc[X_imp_campaign_test.index, 'device'] = y_imp_campaign_test

In [ ]:
imputer = SimpleImputer(missing_values = "na", strategy = 'most_frequent') # Impute the mode for `job` and `education``
campaign_test[['job', 'education']] = imputer.fit_transform(campaign_test[['job', 'education']])

In [ ]:
campaign_test = pd.get_dummies(campaign_test, columns = ['job']) # Dummify the 11 jobs, creating 11 new columns

In [ ]:
enc = OrdinalEncoder(categories = [['high_school', 'university', 'grad_school']]) # Create an ordinal value for `education` 
campaign_test['education'] = enc.fit_transform(campaign_test[['education']])

In [ ]:
campaign_test.to_csv('TEST_KAGGLE.csv') # Export CSV 